In [1]:

import tensorflow as tf
import os
from preprocess import create_data_generators, get_class_map  

print("TensorFlow version:", tf._version_)

ModuleNotFoundError: No module named 'preprocess'

In [ ]:
# Constants
IMG_SIZE = 224
BATCH_SIZE = 32

# Paths 
BASE_DIR = "../dataset" 
TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR = os.path.join(BASE_DIR, "test")

print(f"Checking if paths exist: {os.path.exists(TRAIN_DIR)}")

Checking if paths exist: True


In [ ]:
# Map integer indices to human-readable names
labels = {v: k for k, v in train_generator.class_indices.items()}
print("\nClass Map:", labels)


Class Map: {0: 'glioma_tumor', 1: 'meningioma_tumor', 2: 'no_tumor', 3: 'pituitary_tumor'}


In [ ]:
# Load data generators
ttrain_generator, test_generator = create_data_generators(TRAIN_DIR, TEST_DIR, IMG_SIZE, BATCH_SIZE)

NameError: name 'create_data_generators' is not defined

In [ ]:
labels = get_class_map(train_generator)
print("\nClass Map:", labels)
